# Iceberg experiment analysis script

In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

# Average melting calculations and plots

In [ ]:
longname={'U':'Flow speed U [cm/s]',
          't':'Duration [min]',
          'M0':'Mass initial M0 [g]',
          'M1': 'Mass final M1 [g]',
          'L0': 'Length initial L0 [cm]',
          'W0':'Width initial W0 [cm]',
          'D0':'Depth initial D0 [cm]',
          'dxf':'Front melt [cm]',
          'dxb':'Rear melt [cm]',
          'dy':'Side melt [cm]',
          'dz':'Basal melt [cm]',
          'T':'Temperature T [C]',}
data = pd.read_csv('experiment-data.csv')
shortname={value:key for key,value in longname.items()}
data = data.rename(columns=shortname)

## Average side-dependent melt rates

In [ ]:
data['uf'] = data['dxf']/data['t']
data['ub'] = data['dxb']/data['t']
data['v'] = data['dy']/data['t']
data['w'] = data['dz']/data['t']

In [ ]:
averages = {}
stds = {}
for vel in [0,1.5,3.5]:
    averages[vel] = {}
    stds[vel] = {}
    for melt_rate in ['uf','ub','v','w']:
        melt_rates = data[melt_rate][data['U'] == vel]
        averages[vel][melt_rate] = melt_rates.mean()
        stds[vel][melt_rate] = melt_rates.std()

In [ ]:
np.array([[averages[vel][face] for face in ['uf','v','ub','w']] for vel in [3.5,1.5,0]])

In [ ]:
# Uncertainty defined to be twice standard deviation
np.array([[2*stds[vel][face] for face in ['uf','v','ub','w']] for vel in [3.5,1.5,0]])

In [ ]:
for melt_rate in ['uf','ub','v','w']:
    data[f'{melt_rate} avg'] = [averages[U][melt_rate] for U in data['U'].values]
    data[f'{melt_rate} std'] = [stds[U][melt_rate] for U in data['U'].values]

## Volume loss error calculation

In [ ]:
M0, M1 = data['M0'], data['M1']
ρi = .9167
data['ΔV'] = (M0 - M1)/ρi
L,W,D,uf,ub,v,w,t = [data[field] for field in ['L0','W0','D0','uf','ub','v','w','t']]
uf_avg, ub_avg, v_avg, w_avg, uf_std, ub_std, v_std, w_std = [data[field] for field in ['uf avg','ub avg','v avg','w avg','uf std','ub std','v std','w std']]
uf_std = 2*uf_std.copy()
ub_std = 2*ub_std.copy()
v_std = 2*v_std.copy()
w_std = 2*w_std.copy()
data['ΔV_photo'] = L*W*D - (L-(uf+ub)*t)*(W-2*v*t)*(D-w*t)
data['ΔV_avg'] = L*W*D - (L-(uf_avg+ub_avg)*t)*(W-2*v_avg*t)*(D-w_avg*t)
dV_high = L*W*D - (L-(uf_avg+ub_avg - uf_std-ub_std)*t)*(W-(2*(v_avg-v_std)))*(D-(w_avg-w_std))
dV_low  = L*W*D - (L-(uf_avg+ub_avg + uf_std+ub_std)*t)*(W-(2*(v_avg+v_std)))*(D-(w_avg+w_std))
ddV = dV_high - dV_low

## Melt rate scatter plots and volume loss error plot

In [ ]:
vel_colors = {0:'C0',1.5:'C2',3.5:'C3'}
facecolors = {'uf':'orange','ub':'brown','v':'purple','w':'k'}
vfaces = ['uf','ub','v','w']
markers = {'uf':'<','ub':'>','v':'o','w':'v'}
save_faces = {'uf':'dx f ','ub':'dx b ','v':'dy ','w':'dz '}
frac = {'uf':1,'ub':1,'v':1,'w':1}
labels = {'uf':'Front melt rate $v_f$','ub':'Rear melt rate $v_r$','v':'Side melt rate $v_s$','w':'Basal melt rate $v_b$'}

fig, ax = plt.subplots(1,4,figsize=(10,2.7))#,gridspec_kw={'wspace':.55})
upper = 1700
ax[3].plot([0,upper],[0,upper],'k--')
for i,speed in enumerate([3.5,1.5,0]):
    U = data['U']
    aspect = (L/D)[U==speed]
    for jf, face in enumerate(['uf','ub','v','w']):
        ones = np.ones_like(aspect)
        avg = averages[speed][face]
        std = stds[speed][face]
        ax[i].scatter(aspect, data[face][U==speed],color=facecolors[face],marker=markers[face],label=labels[face],zorder=10-jf)
        ax[i].plot(np.linspace(0,15,len(ones)),avg*ones,'-',color=facecolors[face],linewidth=2)
        ax[i].fill_between(np.linspace(0,15,len(ones)),avg*ones-2*std,avg*ones+2*std,color=facecolors[face],alpha=.2)
    ax[3].scatter(data['ΔV'][U==speed],
                   data['ΔV_avg'][U==speed],
                   s=20,
                   color=vel_colors[speed],
                   label=f'${speed:.1f}$ cm/s')
    ax[3].errorbar(data['ΔV'][U==speed],
                   data['ΔV_avg'][U==speed],
                   fmt='',
                   ls='none',
                   markersize=10,
                   yerr=ddV[U==speed], 
                   color=vel_colors[speed])

ax[0].set(xlim=[2,13],ylim=[0,.5],title='$U = 3.5$ cm s$^{-1}$',xlabel='Aspect ratio $L/D$',ylabel='Melt rate cm min$^{-1}$')
ax[1].set(xlim=[2,13],ylim=[0,.5],title='$U = 1.5$ cm s$^{-1}$',xlabel='Aspect ratio $L/D$',ylabel='Melt rate cm min$^{-1}$')
ax[2].set(xlim=[2,13],ylim=[0,.5],title='$U = 0.0$ cm s$^{-1}$',xlabel='Aspect ratio $L/D$',ylabel='Melt rate cm min$^{-1}$')
ax[2].legend(frameon=False,fontsize=9,borderaxespad=0,loc='upper left')
ax[3].legend(frameon=False,fontsize=9,borderaxespad=0,loc='lower right')
ax[3].set(xlim=[0,upper],ylim=[0,upper],yticks=[0,500,1000,1500],title='Volume loss estimate',
          xlabel='Actual $\Delta V$ cm$^3$',ylabel='Estimated $\Delta V$ cm$^3$')
plt.tight_layout()
plt.savefig('experiment-melt-rates-and-errors.pdf',bbox_inches='tight')

## WC model estimates

In [ ]:
ρ_i = 0.9167 # g/cm^3
ρ_w = 1.021 # g/cm^3
T_i = 0 # C
T_w = 20 # C
L_i = np.array([10,32.5])[None,:] # cm
U_w = np.array([3.5,1.5,0])[:,None] # cm/s
ν = 1.004e-2#1.002e-2 # cm^2/s (at 20 C)
κ = 1.42e-3 # cm^2/s (at 20 c)
c_p = 4.182 # J/(gC)
Λ = 3.34e2 # J/g
coeff = 0.037*(ρ_w/ρ_i)*(ν**(-7/15))*(κ**(2/3))*c_p/Λ
#coeff = 0.75 * (0.15*10/(0.75*2.5**(0.8)*35)) #* 100**(0.4) # cm^0.4 s^-0.2 From FitzMaurice et al. 2017
melt_rates = coeff*(U_w**.8)*(T_w-T_i)/(L_i**0.2) # cm/s
melt_rates *= 60 # cm/min
melt_rates
# coeff

## Holland & Jenkins model estimates

\begin{align}
v\frac{\rho_i}{\rho_w}\frac{\Lambda + c_i (T_b - T_i)}{c_w} - C_d^{1/2} U \Gamma^T (T_w - T_b) &= 0\\
v \frac{\rho_i}{\rho_w} C_b - C_d^{1/2} U \Gamma^C (C_w - C_b) &= 0\\
T_b - \lambda_1 C_b + \lambda_2 &= 0
\end{align}

In [ ]:
ρ_i = 0.9167 # g/cm^3
ρ_w = 1.021 # g/cm^3
T_i = -15 # C
T_w = 20 # C
C_w = 30 # g/kg
L_i = np.array([10,32.5])[None,:] # cm
U_w = np.array([3.5,1.5,0])[:,None] # cm/s
ν = 1.004e-2#1.002e-2 # cm^2/s (at 20 C)
κ = 1.42e-3 # cm^2/s (at 20 c)
c_pw = 4.182 # J/(gC) water heat capacity
c_pi = 2.108 # J/(gC) ice ""
Λ = 3.34e2 # J/g
Cd = 0.0097 # Dimensionless drag coefficient, table 2 Jenkins et al. "Observation and Parameterization of Ablation at the Base of Ronne Ice Shelf, Antarctica" 2010
ΓT = 0.011
ΓC = 3.1e-4
λ1 = -0.057 # C kg / g
λ2 = 0.083 # C
v0 = 0.1/60 # cm/s -- same as U
C_b0 = C_w
T_b0 = λ1*C_b0 + λ2


In [ ]:
from scipy.optimize import newton
from scipy.optimize import fsolve
def HJ_residual(guess,U):
    v,T_b,C_b = guess
    return np.array([v*(ρ_i/ρ_w)*((Λ + c_pi*(T_b-T_i))/c_pw) - Cd**(1/2)*U*ΓT*(T_w-T_b),
                     v*(ρ_i/ρ_w)*C_b - Cd**(1/2)*U*ΓC*(C_w - C_b),
                     T_b - λ1*C_b - λ2])

In [ ]:
v, T_b, C_b = fsolve(HJ_residual,
       [v0,T_b0,C_b0],
       args=(1.5,))
print('v {:.3f}, T_b {:.2f}, C_b {:.2f}'.format(v*60,T_b,C_b))

# Melting profile calculations

In [ ]:
import file_tools as flt
from scipy.interpolate import interp1d
new_file = 'image_analysis.h5'

In [ ]:
# Let's make a plot of the left profiles first

In [ ]:
def depth_profile(im_data,exp):
    # take in picture profiles in pixels, and spline of left/right average depth in cm
    expstr = f'{exp:0>2d}'
    ratiol = np.asscalar(im_data[f'{expstr}/l/dims/cm_pix'][...])
    ratior = np.asscalar(im_data[f'{expstr}/r/dims/cm_pix'][...])

    xl = im_data[expstr]['l']['dims']['x'][...]
    xr = im_data[expstr]['r']['dims']['x'][...]
    topl = im_data[expstr]['l']['dims']['t'][...]
    topr = im_data[expstr]['r']['dims']['t'][...][::-1]
    depthl = im_data[expstr]['l']['dims']['bottom'][...]
    depthr = im_data[expstr]['r']['dims']['bottom'][...][::-1]
    x0l = np.where(np.isfinite(depthl))[0][0]
    x0r = np.where(np.isfinite(depthr))[0][0]

    Xl = ratiol*(xl - x0l)[xl >= x0l]
    Xr = ratior*(xr - x0r)[xr >= x0r]
    Bottoml = ratiol*(depthl - topl)[xl >= x0l]
    Bottomr = ratior*(depthr - topr)[xr >= x0r]
    Bottoml[np.isnan(Bottoml)] = 0
    Bottoml[Bottoml < 0] = 0
    Bottomr[np.isnan(Bottomr)] = 0
    Bottomr[Bottomr < 0] = 0

    Bottomlf = interp1d(Xl, Bottoml)
    Bottomrf = interp1d(Xr, Bottomr)
    h0 = data.iloc[exp]['D0']

    X = np.arange(0, min(Xl[-1],Xr[-1]), .01)
    Bottom_avg = np.mean([Bottomlf(X), Bottomrf(X)], axis=0)

    return X, h0 - Bottom_avg

In [ ]:
cutoffs = {0:[0.01,1,2, 8, 13, 21, 30.5],
           1.5:[0.01,1.5,3.5, 8, 12.5, 19, 24, 31],
           3.5:[0.01,3,5.5, 7.5, 12.5, 19, 24, 31]}

xs = {}
bottoms = {}
avgs = {}

for i, U in enumerate([3.5,1.5,0]):
    xs[U], bottoms[U] = {},{}
    avgs[U] = {k:[] for k in range(len(cutoffs[U])-1)}
    for j, exp in enumerate(data[data['U'] == U].index.values):
        expstr = f'{exp:0>2d}'
        with h5py.File(new_file,'r') as im_data:
            if exp != 9:
                x, bottom = xs[U][j], bottoms[U][j] = depth_profile(im_data,exp)
    
    # calculating average between different profiles
    for j in bottoms[U]:
        x, bottom = xs[U][j], bottoms[U][j]
        for k, cutoff in enumerate(cutoffs[U][1:]):
            amask = (x>cutoffs[U][k])&(x<=cutoff)
            if x[-1] > cutoff:
                avgs[U][k] += [bottom[amask]]
    for k in range(len(cutoffs[U])-1): 
        avgs[U][k] = np.mean(avgs[U][k], axis=0)
    avgs[U] = np.concatenate([avgs[U][k] for k in avgs[U]],)

## Melting profile plots

In [ ]:
def colorfunc(length):
    rescale = .3 + (.9 - .3)* (33 - length)/(33-10)# 10 -> .9, 33 -> .3
    return plt.cm.viridis(rescale)

In [ ]:
import matplotlib.patches as patches

In [ ]:
# plotting
axcs = {}
fig, ax = plt.subplots(3,1,figsize=(10,2.8),gridspec_kw={'hspace':0},sharey=False)
for i, U in enumerate([3.5,1.5,0]):
    colors = plt.cm.viridis(np.linspace(.9,.3,len(xs[U])))
    for ji, j in enumerate(xs[U].keys()):
        x, bottom = xs[U][j], bottoms[U][j]
        plot = ax[i].plot(x, bottom, linewidth=1, color=colorfunc(x[-1]),label='Experiment'*(ji==(len(xs[U])-1)))
    x_av = np.arange(cutoffs[U][0],cutoffs[U][-1],0.01)
    uf, ub, w = averages[U]['uf'], averages[U]['ub'], averages[U]['w']
    duf, dub, dw = 2*stds[U]['uf'], 2*stds[U]['ub'], 2*stds[U]['w']
    u_wc = melt_rates[i].mean()

    rectav = patches.Rectangle(((uf)*10, (w)*10), 32.5-10*(uf+ub), 3.3-10*(w),linewidth=1.5,
                               edgecolor='k',facecolor='none',alpha=1,zorder=3,label='Average')
    ax[i].add_patch(rectav)
    rect1 = patches.Rectangle(((uf-duf)*10, (w-dw)*10), 32.5-10*(uf+ub-duf-dub), 3.3-10*(w-dw),
                              linewidth=1,edgecolor='none',facecolor='k',alpha=0.3,zorder=-10)
    ax[i].add_patch(rect1)
    rect2 = patches.Rectangle(((uf+duf)*10, (w+dw)*10), 32.5-10*(uf+ub+duf+dub), 3.3-10*(w+dw),
                              linewidth=1,edgecolor='none',facecolor='white',alpha=1,zorder=-5)
    ax[i].add_patch(rect2)
    rectWC = patches.Rectangle(((u_wc)*10, (u_wc)*10), 32.5-10*(2*u_wc), 3.3-10*(u_wc),
                               linewidth=1,edgecolor='red',facecolor='none',alpha=1,zorder=3,label='WC model')
    ax[i].add_patch(rectWC)
    
    ax[i].set(xlim=[0,32.5],ylim=[0,3],aspect=1)
    ax[i].annotate(f'$U = {U:.1f}$\n cm s$^{{-1}}$',(1.05,0.2),xycoords='axes fraction',horizontalalignment='right',rotation=-90)
ax[0].set(title='Final side profiles cm',xticks=[],yticks=[0,1,2,3])
ax[1].set(ylabel='Depth $z$ cm',xticks=[],yticks=[0,1,2])
ax[2].set(xlabel='Length $x$ cm',yticks=[0,1,2])
ax[2].legend(loc='lower left',bbox_to_anchor=(.82,0.33),borderaxespad=0,frameon=False,fontsize=7)
plt.savefig('experiment-profiles-averages.pdf',bbox_inches='tight')

# Experimental time series

In [ ]:
import glob
from PIL import Image

exps = ['fast-long','fast-short','slow-long','no-long']

images = {exp:sorted(glob.glob(f'snapshots/{exp}/*')) for exp in exps}

slices = {'fast-long':np.s_[125:300,:],
          'fast-short':np.s_[280:430,450:900],
          'slow-long':np.s_[50:50+150,:],
          'no-long':np.s_[10:,55:-55],}

imgs = {}
for exp in exps:
    imgs[exp] = {}
    for i, image in enumerate(images[exp]):
        imgs[exp][i] = np.array(Image.open(image))[slices[exp]]
aspect_ratios = {}
for exp in exps:
    start_img = imgs[exp][0]
    h, w, _ = shape = start_img.shape
    aspect_ratios[exp] = w/h

## Fast experiments

In [ ]:
fig, ax = plt.subplots(5,2,figsize=(7.3,3.5),
            gridspec_kw={'width_ratios':[aspect_ratios['fast-short'],aspect_ratios['fast-long']],'hspace':0,'wspace':0.01})
ax_outer = fig.add_subplot(frameon=False)
ax_outer.set(xticks=[],yticks=[],title='Time series $U=3.5$ cm s$^{-1}$')
for i, exp in enumerate(['fast-short','fast-long']):
    for j in imgs[exp]:
        ax[j,i].imshow(imgs[exp][j])
        ax[j,i].set(xticks=[],yticks=[])
ax[4,0].set(xlabel='$L = 10$ cm')
ax[4,1].set(xlabel='$L = 32.5$ cm')
for j in range(5): ax[j,0].set(ylabel=f'{2*j} min')
plt.savefig('fast-time-series.png',dpi=500,bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(5,2,figsize=(8,3),gridspec_kw={
    'width_ratios':[aspect_ratios['slow-long'],aspect_ratios['no-long']],'hspace':0,'wspace':0.03})
for i, exp in enumerate(['slow-long','no-long']):
    for j in imgs[exp]:
        ax[j,i].imshow(imgs[exp][j])
        ax[j,i].set(xticks=[],yticks=[])
ax[0,0].set(title='Time series $U = 1.5$ cm s$^{-1}$')
ax[0,1].set(title='Time series $U = 0.0$ cm s$^{-1}$')
ax[4,0].set(xlabel='$L = 32.5$ cm')
ax[4,1].set(xlabel='$L = 32.5$ cm')
for j in range(5): ax[j,0].set(ylabel=f'{2*j} min')
plt.savefig('slow-time-series.png',dpi=500,bbox_inches='tight')

# Geometric model of melting

Plot the geometrically averaged melt rate as a function of aspect ratio for experimental averaged melt rates.

In [ ]:
D = 1
Lmax = 50
L = np.linspace(1,Lmax,101)
W = L
melt_rates = averages
A = 2*(L+W)*D + L*W
dVdt = {}
for U in melt_rates:
    vf, vr, vs, vb = [melt_rates[U][face] for face in ['uf','ub','v','w']]
    dVdt[U] = ((vf+vr)*W*D + 2*vs*L*D + vb*L*W)/A

In [ ]:
colors = {0:'C0',1.5:'C2',3.5:'C3'}
fig, ax = plt.subplots(figsize=(5,3))
for U in [3.5,1.5,0]:
    ax.plot(L, dVdt[U], label=f'$U = {U:.1f}$ cm s$^{{-1}}$', color=colors[U])
    ax.plot(L, np.ones_like(L)*melt_rates[U]['w'],'--',color=colors[U],zorder=-1)
ax.annotate('$v_b = 0.13$ cm min$^{-1}$',(.6,.44),xycoords='axes fraction',color=colors[3.5])
ax.annotate('$v_b = 0.080$ cm min$^{-1}$',(.6,.22),xycoords='axes fraction',color=colors[1.5])
ax.annotate('$v_b = 0.077$ cm min$^{-1}$',(.6,.13),xycoords='axes fraction',color=colors[0])
ax.set(xlim=[0,Lmax],ylim=[0,0.25],xlabel='Aspect ratio $L/D = W/D$',
       ylabel='Melt rate $v_{av}$ cm min$^{-1}$',
       title='Geometrically averaged melt rate $v_a$ cm min$^{-1}$')
ax.legend(framealpha=1,frameon=False)
plt.savefig('geometrically-averaged-melt-rate.pdf',bbox_inches='tight')